In [1]:
import numpy as np
import pandas as pd

## 1. Getting the data.

We'll be using the file 'processed.cleveland.data', which is from https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/. It should already be in the .zip file you downloaded.

For more information abouot the dataset, see the description here: https://archive.ics.uci.edu/ml/datasets/heart+Disease and here: https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names

Run these cells to save the data into a numpy array.

In [2]:
columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]

In [3]:
data = np.genfromtxt("processed.cleveland.data", delimiter=",")

In [4]:
data = np.delete(data, np.where(np.isnan(data).any(axis=1))[0], axis=0)

In [5]:
data_df = pd.DataFrame(data, columns=columns)

Take a look at the dataset:

In [6]:
data_df.head

<bound method NDFrame.head of       age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0    63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1    67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2    67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3    37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4    41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   
5    56.0  1.0  2.0     120.0  236.0  0.0      0.0    178.0    0.0      0.8   
6    62.0  0.0  4.0     140.0  268.0  0.0      2.0    160.0    0.0      3.6   
7    57.0  0.0  4.0     120.0  354.0  0.0      0.0    163.0    1.0      0.6   
8    63.0  1.0  4.0     130.0  254.0  0.0      2.0    147.0    0.0      1.4   
9    53.0  1.0  4.0     140.0  203.0  1.0      2.0    155.0    1.0      3.1   
10   57.0  1.0  4.0     140.0  192.0  0.0      0.0    148.0    0.0      0.4   
11   56.0  0.0  2.0   

In [7]:
print(data.shape)

(297, 14)


In [8]:
data.shape

(297, 14)

**How many examples are in this dataset?**

297

**What are the possible labels?**

In [9]:
labels = [int(label) for label in data[:,-1]]

In [10]:
np.unique(labels)

array([0, 1, 2, 3, 4])

**How many of each label are there?**

In [11]:
for label in np.unique(labels):
    print("number of label %d: %d" % (label, np.count_nonzero(labels == label)))

number of label 0: 160
number of label 1: 54
number of label 2: 35
number of label 3: 35
number of label 4: 13


Read the Data Set Information section here: https://archive.ics.uci.edu/ml/datasets/heart+Disease. **What are we trying to classify?**

Now, we'll make a simplified dataset based on our desired classification.

In [12]:
labels = data[:,-1]
labels[labels > 0] = 1
data[:,-1] = labels

**Now what are the possible labels? How many of each label do we have? What do each of these labels mean?**

In [13]:
labels = [int(label) for label in data[:,-1]]
np.unique(labels)

array([0, 1])

In [14]:
for label in np.unique(labels):
    print("number of label %d: %d" % (label, np.count_nonzero(labels == label)))

number of label 0: 160
number of label 1: 137


A label of 0 means no heart disease, and a label of 1 means heart disease.

**What is the chance rate if we randomly guess a label for each input? What if we always guess the most common label?**

Random guessing:

In [15]:
1.0/2

0.5

Guessing most common label:

In [16]:
160/len(labels)

0.5387205387205387

**What accuracy do you think we'll be able to get using K-NN or a decision tree?**

## 2. Splitting into train and test sets.

Run the following cell to split our data into a train and test set. We'll start by using 80% of our data for the train set and 20% for the test set. If you have time, feel free to experiment with different train-test splits.

In [17]:
num_examples = data.shape[0]
indices = np.random.permutation(num_examples)

train_indices = indices[:int(num_examples * 4 / 5)]
test_indices = indices[int(num_examples * 4 / 5):]

train_data = data[train_indices]
test_data = data[test_indices]

**How many train and test examples do we have?**

In [18]:
print(train_data.shape)

(237, 14)


In [19]:
print(test_data.shape)

(60, 14)


Which columns contain each example's features, and which column contains the label? What are the attributes? (hint: see https://archive.ics.uci.edu/ml/datasets/heart+Disease for more information about the dataset.)

## 3. Implementing k-Nearest Neighbors.

### 3a. Nearest Neighbors.

First, we'll need to write some functions to compute distance. Fill in the following functions for euclidean and manhattan distance, which we discussed in today's lecture.

In [20]:
def euclidean_distance(example1, example2):
    squared_diff_sum = 0
    for example1_feature, example2_feature in zip(example1, example2):
        squared_diff_sum += (example1_feature - example2_feature) ** 2
    euclidean_distance = np.sqrt(squared_diff_sum)
    return euclidean_distance

In [21]:
def manhattan_distance(example1, example2):
    diff_sum = 0
    for example1_feature, example2_feature in zip(example1, example2):
        diff_sum += abs(example1_feature - example2_feature)
    return diff_sum

Next, we'll write a function to do nearest neighbors classification.

In [22]:
def nearest_neighbor_classifier(train_inputs, train_labels, test_input):
    closest_index = -1
    smallest_distance = np.inf
    train_size = len(train_inputs)
    for train_index in range(train_size):
        train_input = train_inputs[train_index]
        distance = euclidean_distance(train_input, test_input)
        if distance < smallest_distance:
            closest_index = train_index
            smallest_distance = distance
    predicted_label = train_labels[closest_index]
    return predicted_label

Now fill in the following function, which will let us test our predictor's accuracy.

In [23]:
def test_predictor_accuracy(train_data, test_data):
    train_inputs = train_data[:,:-1]
    train_labels = train_data[:,-1]
    test_inputs = test_data[:,:-1]
    test_labels = test_data[:,-1]
    test_size = test_inputs.shape[0]
    correct_predictions = 0
    for test_index in range(test_size):
        test_input = test_inputs[test_index]
        test_label = test_labels[test_index]
        prediction = nearest_neighbor_classifier(train_inputs, train_labels, test_input)
        if prediction == test_label:
            correct_predictions += 1
    accuracy = correct_predictions / test_size
    return accuracy

In [24]:
test_predictor_accuracy(train_data, test_data)

0.6333333333333333

**Is this accuracy higher or lower than what you would have expected?**

### 3b. k-Nearest Neighbors.

Now let's try extending our function to implement k-nearest neighbors. First, we'll need a function that finds the most common label in a list of labels.

In [25]:
def majority_vote(labels):
    unique_labels = np.unique(labels)
    num_unique_labels = len(unique_labels)
    unique_labels_counts = {label:0 for label in unique_labels}
    for label in labels:
        unique_labels_counts[label] += 1
    most_common_label = list(unique_labels_counts.keys())[np.argmax(list(unique_labels_counts.values()))]
    return most_common_label

Now fill in the following function to implement k-nearest neighbors.

In [26]:
def k_nearest_neighbor_classifier(train_inputs, train_labels, test_input, k):
    train_size = len(train_inputs)
    distances = np.zeros(train_size)
    for train_index in range(train_size):
        train_input = train_inputs[train_index]
        distance = euclidean_distance(train_input, test_input)
        distances[train_index] = distance
    distance_order = np.argpartition(distances, k)
    k_closest_train_indices = distance_order[:k]
    k_closest_train_labels = train_labels[k_closest_train_indices]
    predicted_label = majority_vote(k_closest_train_labels)
    return predicted_label

Finally, fill in the following function to test our predictor's accuracy.

In [27]:
def test_knn_predictor_accuracy(train_data, test_data, k):
    train_inputs = train_data[:,:-1]
    train_labels = train_data[:,-1]
    test_inputs = test_data[:,:-1]
    test_labels = test_data[:,-1]
    test_size = test_inputs.shape[0]
    correct_predictions = 0
    for test_index in range(test_size):
        test_input = test_inputs[test_index]
        test_label = test_labels[test_index]
        prediction = k_nearest_neighbor_classifier(train_inputs, train_labels, test_input, k)
        if prediction == test_label:
            correct_predictions += 1
    accuracy = correct_predictions / test_size
    return accuracy

What is our predictor's accuracy?

In [28]:
test_knn_predictor_accuracy(train_data, test_data, 1)

0.6333333333333333

### 3c. Nearest neighbors using sklearn.

We just implemented nearest neighbors by ourself, but we can also borrow code that other people have written. `sklearn` has a lot of functions useful for machine learning, and we'll try out their function for nearest neighbors.

In [29]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
train_inputs = train_data[:,:-1]
train_labels = train_data[:,-1]
neigh.fit(train_inputs, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [30]:
test_inputs = test_data[:,:-1]
test_labels = test_data[:,-1]
neigh.score(test_inputs, test_labels)

0.6333333333333333

**Do you get the same accuracies as you did for your own implementation?**

## 4. Decision Trees.

Now, we'll try using a decision tree classifier. We'll use sklearn again here.

### 4a: Building and training the tree.
First, we need to build the decision tree and fit it to our train data.

In [31]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_inputs, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

### 4b. Testing the tree.
Now let's test our classifier on the test data.

In [32]:
clf.score(test_inputs, test_labels)

0.66666666666666663

### 4c. Viewing the tree.
If you're interested in seeing what the tree actually looks like, run the following cells. The tree will be saved in the file test.pdf.

In [33]:
from sklearn.tree import export_graphviz
import graphviz

In [34]:
import os
os.environ["PATH"] += os.pathsep +'C:/Users/Cathy/Anaconda3/Library/bin/graphviz/'

In [35]:
dot_data = export_graphviz(clf, out_file=None, class_names=True, label='none', impurity=False) 
graph = graphviz.Source(dot_data) 
graph.render("test") 

'test.pdf'

### Extra Challenge.

"Extra challenge" sections are a more unguided exploration into the concepts we've discussed. You'll notice less scaffolding for the code -- try implementing these concepts from scratch, and feel free to ask your neighbors or an instructor if you have any questions!

Now that we've created some nearest neighbors classifiers, see how changing certain aspects of the classifier changes the classification accuracy. Some ideas of things to try:
- Change the train/test split..
- Try different values if $k$ -- what seems to give the best classification accuracy?
- Test both the euclidean and manhattan distance functions -- does one give a better classification accuracy than the other?